In [3]:
import pandas as pd
import base64

In [16]:
watch = [
    [
        'Elliot Page',
        'RWxsZW4=',
        'Elliot_Page',
        '2020-12-01'
    ],
    [
        'Chelsea Manning',
        'QnJhZGxleQ==',
        'Chelsea_Manning',
        '2012-08-22'
    ],
    [
        'Elke Mackenzie',
        'SXZhbg==',
        'Ivan_Mackenzie_Lamb',
        None
    ],
    [
        'Jennell Jaquays',
        'UGF1bCBKYXF1YXlz',
        'Jennell_Jaquays',
        '2021-03-04'
    ],
    [
        'Wendy Carlos',
        'V2FsdGVy',
        'Wendy_Carlos',
        None
    ],
    [
        'Andréa Furet',
        'QWxleGFuZHJl',
        'Andréa_Furet',
        '2019-06-01'
    ],
]

watch

[['Elliot Page', 'RWxsZW4=', 'Elliot_Page', '2020-12-01'],
 ['Chelsea Manning', 'QnJhZGxleQ==', 'Chelsea_Manning', '2012-08-22'],
 ['Elke Mackenzie', 'SXZhbg==', 'Ivan_Mackenzie_Lamb', None],
 ['Jennell Jaquays', 'UGF1bCBKYXF1YXlz', 'Jennell_Jaquays', '2021-03-04'],
 ['Wendy Carlos', 'V2FsdGVy', 'Wendy_Carlos', None],
 ['Andréa Furet', 'QWxleGFuZHJl', 'Andréa_Furet', '2019-06-01']]

In [17]:
def decode(string):
    return base64.b64decode(string).decode("utf-8")

In [33]:
def load(i):
    page_name = i[2]
    
    main = (
        pd
        .read_csv(f'../data/page-revisions/{page_name.lower()}.csv')
        .set_index('revid')
        .assign(page = page_name)
        .assign(
            deadname_count = lambda df: df.added.apply(lambda x: ' '.join(eval(x)).lower().count(decode(i[1]).lower()))
        )
    )
    
    talk = (
        pd
        .read_csv(f'../data/page-revisions/discussion:{page_name.lower()}.csv')
        .set_index('revid')
        .assign(page = f'discussion:{page_name}')
        .assign(
            deadname_count = lambda df: df.added.apply(lambda x: ' '.join(eval(x)).lower().count(decode(i[1]).lower()))
        )
    )
    
    df = pd.concat([main, talk])
    return df

df = pd.concat([ load(i) for i in watch])

In [40]:
users = (
    df
    .query('timestamp >= "2024-01-15"')
    .query('deadname_count > 0')
    .groupby('user')
    .agg({
        'deadname_count': 'sum'
    })
    .sort_values('deadname_count', ascending=False)
)

users

,deadname_count
user,
Sherwood6,73
GF38storic,43
Cymbella,16
Triboulet sur une montagne,9
Lewisiscrazy,6
Anne Daemon,6
Enthalpist,6
JohnNewton8,4
Lomita,3


In [48]:
(
    df
    .query('timestamp >= "2024-01-15"')
    .query('user.isin(@users.head(4).index)')
    .query('deadname_count > 0')
    [['user', 'deadname_count', 'page', 'timestamp']]
    .to_csv('../data/deadnamers.csv')
)